# Select spot dection thresholds

To set up base calling, we need to create bleed-through matrices. This is done by 
selecting a threshold for spot detection and one for filtering isolated spots. This 
notebook will help you select the best threshold for your data.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy

import iss_preprocess as iss

data_path = "becalia_rabies_barseq/BRAC8501.6a/chamber_07"
prefix = "barcode_round"
ref_tile_index = 1  # which of the reference tiles do we want to use for plots

In [ ]:
ops = iss.io.load.load_ops(data_path)
short_pref = prefix.split("_")[0]
ref_tiles = ops[f"{short_pref}_ref_tiles"]
print(f"{len(ref_tiles)} reference tiles found. Using {ref_tile_index}.")
ref_tile = ref_tiles[ref_tile_index]

In [ ]:
print(f"Loading registered data for {ref_tile}")
stack, _ = iss.pipeline.load_and_register_sequencing_tile(
    data_path,
    ref_tile,
    filter_r=ops["filter_r"],
    prefix=prefix,
    suffix=ops[f"{short_pref}_projection"],
    nrounds=ops[f"{prefix}s"],
    correct_channels=ops[f"{short_pref}_correct_channels"],
    corrected_shifts=ops["corrected_shifts"],
    correct_illumination=False,
)
stack = stack[:, :, np.argsort(ops["camera_order"]), :]

In [ ]:
print("Making reference image using STD")
reference = np.std(stack, axis=(2, 3))
reference.shape

## Spot detection

First step is to detect some spots

In [ ]:
from iss_preprocess.segment.spots import detect_spots

detection_threshold = (ops[f"{short_pref}_detection_threshold"],)
spots = detect_spots(reference, threshold=detection_threshold)

In [ ]:
from iss_preprocess.vis import plot_matrix_with_colorbar, round_to_rgb

center = np.array(stack.shape[:2]) // 2
w = 200
extent = [[center[1] - w, center[1] + w], [center[0] - w, center[0] + w]]

valid_spots = spots[
    (spots.x > extent[1][0])
    & (spots.x < extent[1][1])
    & (spots.y > extent[0][0])
    & (spots.y < extent[0][1])
]

rgb = round_to_rgb(
    stack,
    1,
    extent=extent,
    channel_colors=([1, 0, 0], [0, 1, 0], [1, 0, 1], [0, 1, 1]),
    vmax=np.percentile(stack[..., 0], 99.99, axis=(0, 1)),
    vmin=np.percentile(stack[..., 0], 0.2, axis=(0, 1)),
)
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].imshow(rgb)
axes[0].set_title("Raw data, round 1")
ref_part = reference[slice(*extent[0]), slice(*extent[1])]
cax, cb = plot_matrix_with_colorbar(
    ref_part,
    axes[1],
    cmap="viridis",
    vmax=np.percentile(reference, 99.999),
    vmin=np.percentile(reference, 0.2),
)
axes[1].contour(ref_part, levels=[detection_threshold], colors="r", linewidths=0.5)
cax.axhline(detection_threshold, color="r", lw=2)
axes[1].set_title("Standard deviation projection")
for ax in axes:
    ax.scatter(
        valid_spots.x - extent[1][0],
        valid_spots.y - extent[0][0],
        s=2,
        c="k",
        marker="x",
    )
    ax.axis("off")
fig.tight_layout()

## Select isolated spots

We keep only spots with a minimum distance to the next spot. This is done by mesuring
the fluorescence in an annulus around each spot and thresholding

In [ ]:
from iss_preprocess.coppafish import annulus
from iss_preprocess.segment import detect_isolated_spots

isolation_threshold = ops[f"{short_pref}_isolation_threshold"]
annulus_r = (3, 7)
strel = annulus(annulus_r[0], annulus_r[1])
strel = strel / np.sum(strel)
annulus_image = scipy.ndimage.correlate(reference, strel)
isolated = annulus_image[spots["y"], spots["x"]] < isolation_threshold
isolated_spots = spots[isolated]
non_isolated_spots = spots[~isolated]

In [ ]:
annulus_part = annulus_image[slice(*extent[0]), slice(*extent[1])]
fig, axes = plt.subplots(1, 3, figsize=(10, 5))
plot_matrix_with_colorbar(
    ref_part,
    axes[0],
    vmax=np.percentile(ref_part, 99.999),
    vmin=np.percentile(ref_part, 0.2),
)
axes[0].set_title("Standard deviation projection")

cax, cb = plot_matrix_with_colorbar(
    annulus_part,
    axes[1],
    vmax=np.percentile(annulus_part, 99.999),
    vmin=np.percentile(annulus_part, 0.05),
)
print(isolation_threshold)
cax.axhline(isolation_threshold, color="r", lw=2)
if isolation_threshold < cax.get_ylim()[0]:
    cax.set_ylim(isolation_threshold, cax.get_ylim()[1])
if isolation_threshold > cax.get_ylim()[1]:
    cax.set_ylim(cax.get_ylim()[0], isolation_threshold)
for c, w, l in zip(
    ["r", "k"], [non_isolated_spots, isolated_spots], ["non-isolated", "isolated"]
):
    v = w[
        (w.x > extent[1][0])
        & (w.x < extent[1][1])
        & (w.y > extent[0][0])
        & (w.y < extent[0][1])
    ]
    axes[1].scatter(
        v.x - extent[1][0], v.y - extent[0][0], s=2, c=c, marker="+", label=l
    )
axes[1].legend(
    loc="lower left",
    bbox_to_anchor=(0.0, 0.0),
)
axes[1].set_title("Annulus projection")

# plot annulus values only where spots are
img = np.zeros_like(annulus_part) + np.nan
values = annulus_image[spots["y"], spots["x"]]
valid_values = annulus_image[valid_spots["y"], valid_spots["x"]]
for i, spot in valid_spots.iterrows():
    px = int(spot.x - extent[1][0])
    py = int(spot.y - extent[0][0])
    img[py, px] = annulus_part[py, px]
vmin, vmax = valid_values.min(), valid_values.max()
sc = axes[2].scatter(
    valid_spots.x - extent[1][0],
    valid_spots.y - extent[0][0],
    s=2,
    c=valid_values,
    marker="x",
    vmin=vmin,
    vmax=vmax,
)
axes[2].set_title("Annulus projection, spot values")
plot_matrix_with_colorbar(img, axes[2], vmin=vmin, vmax=vmax)

for ax in axes:
    ax.axis("off")

## Extract spots

Now that we have detected isolated spots, we can extract the fluorescence of each spot.
This is done with a given radius around the spot.


In [ ]:
spots = detect_isolated_spots(
    reference,
    detection_threshold=detection_threshold,
    isolation_threshold=isolation_threshold,
)

iss.call.extract_spots(spots, stack, ops["spot_extraction_radius"])

In [ ]:
# plot the spots in the part of reference image and show the extraction radius
from skimage.draw import disk

fig, axes = plt.subplots(1, 2, figsize=(10, 5))
w = 100
smallext = [
    [center[1] - w, center[1] + w],
    [center[0] - w, center[0] + w],
]

valid_spots = spots[
    (spots.x > smallext[1][0])
    & (spots.x < smallext[1][1])
    & (spots.y > smallext[0][0])
    & (spots.y < smallext[0][1])
]

ref_part = reference[slice(*smallext[0]), slice(*smallext[1])]

plot_matrix_with_colorbar(
    ref_part,
    axes[0],
    vmax=np.percentile(ref_part, 99.999),
    vmin=np.percentile(ref_part, 0.2),
)
axes[0].set_title("Standard deviation projection")
axes[0].scatter(
    valid_spots.x - smallext[1][0],
    valid_spots.y - smallext[0][0],
    s=2,
    c="r",
    marker="x",
)

for _, spot in spots.iterrows():
    rr, cc = disk(
        (spot["y"], spot["x"]), ops["spot_extraction_radius"], shape=stack.shape[0:2]
    )
img = np.zeros_like(ref_part) + np.nan
for i, spot in spots.iterrows():
    if (
        spot.x < smallext[1][0]
        or spot.x >= smallext[1][1]
        or spot.y < smallext[0][0]
        or spot.y >= smallext[0][1]
    ):
        continue
    px = int(spot.x - smallext[1][0])
    py = int(spot.y - smallext[0][0])
    img[py, px] = ref_part[py, px]
cmap = plt.cm.viridis
cmap.set_bad("black", 1)
plot_matrix_with_colorbar(
    img,
    axes[1],
    vmax=np.percentile(ref_part, 99.999),
    vmin=np.percentile(ref_part, 0.2),
    cmap=cmap,
    interpolation="none",
)

axes[1].set_title("Extracted pixels")
print(f"Spot extraction radius: {ops['spot_extraction_radius']}.")
for ax in axes:
    ax.axis("off")
fig.tight_layout()